# Informations:
-   the dataframa i using is avilable in https://www.kaggle.com/datasets/mirichoi0218/insurance

Columns

    age: age of primary beneficiary

    sex: insurance contractor gender, female, male

    bmi: Body mass index, providing an understanding of body, weights that are relatively high or low relative to height,
    objective index of body weight (kg / m ^ 2) using the ratio of height to weight, ideally 18.5 to 24.9

    children: Number of children covered by health insurance / Number of dependents

    smoker: Smoking

    region: the beneficiary's residential area in the US, northeast, southeast, southwest, northwest.

    charges: Individual medical costs billed by health insurance


In [5]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import warnings

warnings.filterwarnings('ignore')

In [6]:
df = pd.read_csv("https://raw.githubusercontent.com/stedy/Machine-Learning-with-R-datasets/master/insurance.csv")

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB


In [8]:
df.describe()

,age,bmi,children,charges
count,1338.000000,1338.000000,1338.000000,1338.000000
mean,39.207025,30.663397,1.094918,13270.422265
std,14.049960,6.098187,1.205493,12110.011237
min,18.000000,15.960000,0.000000,1121.873900
25%,27.000000,26.296250,0.000000,4740.287150
50%,39.000000,30.400000,1.000000,9382.033000
75%,51.000000,34.693750,2.000000,16639.912515
max,64.000000,53.130000,5.000000,63770.428010


In [9]:
df.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


### applying one-hot to categorical variables

In [10]:
df1 = pd.get_dummies(df,drop_first=True)

In [11]:
def zero_or_one(x):
    """Returns 1 if x is True, 0 otherwise."""
    return 1 if x else 0
    

def bool_to_numeric(dataframe: pd.DataFrame, col_names: list) -> pd.DataFrame:
  """
  Transforms boolean columns in a DataFrame to numeric (1 or 0).

  Args:
      dataframe: The DataFrame containing boolean columns.
      col_names: A list of column names to convert.

  Returns:
      A new DataFrame with the specified columns converted to numeric.
  """

  dataframe[col_names] = dataframe[col_names].astype(int)
  return dataframe

df1 = bool_to_numeric(df1,df1.columns[4:])

In [12]:
df1

,age,bmi,children,charges,sex_male,smoker_yes,region_northwest,region_southeast,region_southwest
0,19,27.900,0,16884.92400,0,1,0,0,1
1,18,33.770,1,1725.55230,1,0,0,1,0
2,28,33.000,3,4449.46200,1,0,0,1,0
3,33,22.705,0,21984.47061,1,0,1,0,0
4,32,28.880,0,3866.85520,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...
1333,50,30.970,3,10600.54830,1,0,1,0,0
1334,18,31.920,0,2205.98080,0,0,0,0,0
1335,18,36.850,0,1629.83350,0,0,0,1,0
1336,21,25.800,0,2007.94500,0,0,0,0,1


In [9]:
X = df1.drop(columns='charges')
y = df1.charges

In [10]:
X.astype(float).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   age               1338 non-null   float64
 1   bmi               1338 non-null   float64
 2   children          1338 non-null   float64
 3   sex_male          1338 non-null   float64
 4   smoker_yes        1338 non-null   float64
 5   region_northwest  1338 non-null   float64
 6   region_southeast  1338 non-null   float64
 7   region_southwest  1338 non-null   float64
dtypes: float64(8)
memory usage: 83.8 KB


In [12]:
X_train,X_test, y_train, y_test = train_test_split(X.astype(float),
                                                   y.astype(float),
                                                   test_size=0.2)

In [13]:
class Hidden_layers(tf.keras.layers.Layer):
    def __init__(self,units,
                      activation,
                      num_hidden_states,
                      drop_rate,
                      drop_out):
        
        super(Hidden_layers,self).__init__()
        self.unit = units
        self.activation = activation
        self.num_hidden_states = num_hidden_states
        self.drop_out = drop_out
        self.drop_rate = drop_rate

        self.input_layer = tf.keras.layers.Dense(units=self.unit, activation=self.activation, input_shape = (8,)) 
        self.dense = tf.keras.layers.Dense(units=self.unit, activation=self.activation)
        self.dropout = tf.keras.layers.Dropout(self.drop_rate)
    
    def call(self,inputs):
        if self.drop_out:
            input_layer = self.input_layer(inputs)
            drop = self.dropout(input_layer)
            for i in range(self.num_hidden_states):
                dense = self.dense(drop)
                drop = self.dropout(dense)
            return drop
        else:
            dense = self.input_layer(inputs)
            for i in range(self.num_hidden_states):
                dense = self.dense(dense)
            return dense
    

class Regression_model(tf.keras.Model):
    def __init__(self,units,
                      activation,
                      num_hidden_states,
                      drop_rate = 0.2,
                      drop_out = False):
        
        super().__init__()
        self.units = units
        self.activation = activation
        self.num_hidden_states = num_hidden_states
        self.drop_rate = drop_rate
        self.drop_out = drop_out

        self.hiddens_layers = Hidden_layers(self.units,
                                            self.activation,
                                            self.num_hidden_states,
                                            self.drop_rate,
                                            self.drop_out)
        
        self.main_output = tf.keras.layers.Dense(1,name='Output')

    def call(self,inputs):
        hidden_sate = self.hiddens_layers(inputs)
        output = self.main_output(hidden_sate)

        return output

In [14]:
X_train.shape

(1070, 8)

In [15]:
model = Regression_model(units=20,
                         activation='relu',
                         num_hidden_states=3,
                         drop_rate=0.2,
                         drop_out=True)

/home/carlos/anaconda3/envs/tensorflow-park/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2024-05-27 10:09:50.542334: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-27 10:09:50.567893: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/AB

In [16]:
model.summary()

Model: "regression_model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ hidden_layers (Hidden_layers)   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Output (Dense)                  │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [17]:
model.compile(loss=tf.keras.losses.mae,
              optimizer= tf.keras.optimizers.SGD(),
              metrics = ['mae'])

model.fit(X_train,y_train,epochs=10)

Epoch 1/10


I0000 00:00:1716815449.706640   17058 service.cc:145] XLA service 0x792d280058c0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1716815449.706665   17058 service.cc:153]   StreamExecutor device (0): NVIDIA GeForce RTX 4050 Laptop GPU, Compute Capability 8.9
2024-05-27 10:10:49.716451: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-05-27 10:10:49.763840: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8907


 1/34 ━━━━━━━━━━━━━━━━━━━━ 22s 672ms/step - loss: 11798.1914 - mae: 11798.1914

I0000 00:00:1716815450.112220   17058 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 2783859.2500 - mae: 2783859.2500
Epoch 2/10
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 525us/step - loss: 13193.8818 - mae: 13193.8818
Epoch 3/10
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 651us/step - loss: 13673.7217 - mae: 13673.7217
Epoch 4/10
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 654us/step - loss: 13226.6816 - mae: 13226.6816
Epoch 5/10
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 759us/step - loss: 13794.5547 - mae: 13794.5547
Epoch 6/10
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 702us/step - loss: 13388.2920 - mae: 13388.2920
Epoch 7/10
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 610us/step - loss: 13961.4346 - mae: 13961.4346
Epoch 8/10
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 13372.1758 - mae: 13372.1758 
Epoch 9/10
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 532us/step - loss: 13381.9287 - mae: 13381.9287
Epoch 10/10
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 538us/step - loss: 13388.0947 - mae: 13388.0947
